In [ ]:
from spf.dataset.spf_dataset import v5spfdataset
import glob

glob.glob("./[0-9].*")
fns = [
    "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_03_01_17_43_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_11_40_03_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_06_54_50_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_06_32_54_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_06_06_35_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_01_17_43_nRX2_rx_circle.zarr",
]
# fns = glob.glob("/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_*.zarr")

dss = [
    v5spfdataset(
        fn,
        nthetas=65,
        ignore_qc=True,
        precompute_cache="/home/mouse9911/precompute_cache_chunk16_fresh/",
    )
    for fn in fns
]

for fn in fns:
    ds = v5spfdataset(
        fn,
        nthetas=65,
        ignore_qc=True,
        precompute_cache="/home/mouse9911/precompute_cache_chunk16_fresh/",
    )
    print(fn, len(ds))

In [ ]:
# for ds in dss:
#     print(len(ds))

In [ ]:
import numpy as np

from spf.rf import pi_norm


def bin_error_by_theta(ds, nthetas=65):
    bin_step_size = (2 * np.pi) / (nthetas - 1)
    counts = np.zeros((2, nthetas))
    squared_error_summed = np.zeros((2, nthetas))
    for ridx in [0, 1]:
        gt_theta = ds.ground_truth_thetas[ridx]
        gt_phi = ds.ground_truth_phis[ridx]
        mean_phase = ds.mean_phase[f"r{ridx}"]
        for idx in range(gt_theta.shape[0]):
            bin = int((gt_theta[idx] + np.pi) // bin_step_size)
            counts[ridx, bin] += 1
            # squared_error_summed[ridx, bin] += (
            #     pi_norm(gt_phi[idx] - mean_phase[idx]) ** 2
            # )
            squared_error_summed[ridx, bin] += pi_norm(mean_phase[idx]) ** 2
    return counts, squared_error_summed

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for ds in dss[:5]:
    if len(ds) < 1000:
        continue
    full_error_matrix = bin_error_by_theta(ds, nthetas=65)
    error_matrix = np.sqrt(full_error_matrix[1] / full_error_matrix[0])
    error_matrix
    axs[0].plot(error_matrix[0])
    axs[1].plot(error_matrix[1])

In [ ]:
import matplotlib.pyplot as plt

nthetas = 65
full_error_matrix = np.array(
    [bin_error_by_theta(ds, nthetas=nthetas) for ds in dss]
).sum(
    axis=0
)  # nradios x (counts,squared_error_sum),ntheta
error_matrix = np.sqrt(full_error_matrix[1] / full_error_matrix[0])
fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
axs[0].set_title("radio 0")
axs[1].set_title("radio 1")

axs[0].set_xticks(np.linspace(0, nthetas - 1, 5))
axs[0].set_xticklabels(["-pi", "-pi/2", "0", "pi/2", "pi"])

axs[0].set_ylabel("stddev err (units: oh_nos)")
fig.suptitle("error profile of radios (should be same between): range is -pi/2~pi/2")
axs[0].plot(error_matrix[0])
axs[1].plot(error_matrix[1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def f(x):
    return 2.5 * np.exp(-((abs(x) - np.pi / 2) ** 2))


plt.plot(f(np.linspace(-np.pi, np.pi, 128)))
plt.plot(f(np.linspace(-np.pi, np.pi, 128)) ** 2)
plt.ylim([0, None])